In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use(['bmh'])
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,SelectPercentile,mutual_info_regression
import statsmodels.api as sm
from scipy import stats
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
##############################
from IPython.display import SVG
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_curve, roc_auc_score, make_scorer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier, LogisticRegressionCV,LogisticRegression
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import RidgeClassifierCV, LassoCV, LassoLarsCV, LassoLarsIC, Ridge, Lasso
from sklearn import svm, datasets
from scipy.stats import boxcox
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from itertools import product
from sklearn.linear_model import LassoLarsCV
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error,r2_score
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import ElasticNet
import os
from pandas import ExcelWriter
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, adjusted_rand_score
from kneed import KneeLocator

In [2]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

<IPython.core.display.Javascript object>

In [3]:
def stepwise_selection(data, target,SL_in=0.05,SL_out = 0.05):
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<SL_in):
            best_features.append(new_pval.idxmin())
            while(len(best_features)>0):
                best_features_with_constant = sm.add_constant(data[best_features])
                p_values = sm.OLS(target, best_features_with_constant).fit().pvalues[1:]
                max_p_value = p_values.max()
                if(max_p_value >= SL_out):
                    excluded_feature = p_values.idxmax()
                    best_features.remove(excluded_feature)
                else:
                    break 
        else:
            break
    return best_features

In [4]:
def process_data(data_original,data,cv,q,NumofR):
    if 'Unnamed: 0' in data.columns:
        data.rename(columns={'Unnamed: 0':'Link_ID'}, inplace=True)
    data['y_binary']=data_original['y_binary'].values
    data['Yield_per_plant']=data_original['Yield_per_plant'].values
    x_column_list = data.drop(columns=['Link_ID', 'y_binary', 'Yield_per_plant']).columns
    ###Split Data  
    data_train,data_val=train_test_split(data,train_size=0.8, random_state=42)
    y_b = 'y_binary'
    y_c = 'Yield_per_plant'
    percent_label=[round(100*len(np.where(data_train[y_b]==0)[0])/len(data_train)),round(100*len(np.where(data_val[y_b]==0)[0])/len(data_val))]
    #Feature Selection
    feature_list1 = data[x_column_list].max().sort_values(ascending=False)[data[x_column_list].max()>np.quantile(data[x_column_list].max(),q)].index
    #apply SelectKBest class to extract top 20 best features
    bestfeatures = SelectKBest(score_func=mutual_info_regression, k=20)
    fit = bestfeatures.fit(data[x_column_list],data[y_c])
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(x_column_list)
    #concat two dataframes for better visualization 
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Variable','Score']  #naming the dataframe columns
    feature_list2 = featureScores.nlargest(20,'Score')
    feature_list3=stepwise_selection(data.drop(columns=
                                ['Link_ID', 'y_binary', 'Yield_per_plant']),data[y_c])
    reg = LassoCV(cv=cv).fit(data_train[x_column_list], data_train[y_c])
    lassoCoefs0 = pd.DataFrame(
    data=reg.coef_[np.where(reg.coef_ != 0)[0]], 
    index=data[x_column_list].columns[np.where(reg.coef_ != 0)[0]],columns=['LASSO Coefs1'])
    feature_list4=lassoCoefs0
    
    parameters = {'n_estimators':(10,20,100, 500),
                  'min_samples_split':(2,3,4),
                  'min_samples_leaf':(1,2,3)}

    gb_model = GradientBoostingRegressor(random_state=7, warm_start=False)
    grid_obj = GridSearchCV(gb_model, param_grid=parameters, verbose=1, n_jobs=4, cv=cv)
    grid_obj = grid_obj.fit(data_train[x_column_list],data_train[y_c])
    gb_model_best = grid_obj.best_estimator_
    y_hat = gb_model_best.predict(data_val[x_column_list])
    FeatImportance = gb_model_best.feature_importances_
    GBCoefs = pd.DataFrame(index=data_train[x_column_list].columns, data=FeatImportance,columns=['Coefs'])
    imp_coef = GBCoefs.sort_values(by='Coefs')
    feature_list6=imp_coef.loc[imp_coef['Coefs']>np.quantile(imp_coef,0.7)]
    feature_list6 = [item for item in feature_list6.index]
    
    
    
    parameters = {'n_estimators':(10,20,100, 500),
              'min_samples_split':(2,3,4),
              'min_samples_leaf':(1,2,3)}
    rf_model = RandomForestRegressor(warm_start=False)
    grid_obj = GridSearchCV(rf_model, param_grid=parameters, verbose=1, n_jobs=4, cv=cv)
    grid_obj = grid_obj.fit(data_train[x_column_list],data_train[y_c])
    rf_model_best = grid_obj.best_estimator_
    y_hat_rf = rf_model_best.predict(data_val[x_column_list])  
    rf_model_best.score(data_val[x_column_list],data_val[y_c])
    FeatImportance = rf_model_best.feature_importances_
    RFCoefs = pd.DataFrame(index=data_train[x_column_list].columns, data=FeatImportance,columns=['Coefs'])
    imp_coef_RF = RFCoefs.sort_values(by='Coefs')
    feature_list7=imp_coef_RF.loc[imp_coef_RF['Coefs']>np.quantile(imp_coef_RF,0.7)]
    feature_list7 = [item for item in feature_list7.index]
    
    mi = mutual_info_regression(data_train[x_column_list],data_train[y_c], discrete_features=False, n_neighbors=3, copy=True, random_state=None)
    MI=pd.DataFrame(mi)
    MI['OTU']=data[x_column_list].columns
    MI=MI.set_index('OTU')
    MI=MI.sort_values(0,ascending=False)
    feature_list8=MI[MI[0]>np.quantile(mi,q)].index
    
    methodList = ['Maximum', 
              'KBest', 
              'Stepwise Regression', 
              'Lasso CV', 
              'GBM', 
              'Random Forest', 
              'Mutual Info']
    featureList = [[item for item in feature_list1], 
               [item for item in feature_list2['Variable'].values], 
               feature_list3, 
               [item for item in feature_list4.index], 
               feature_list6, 
               feature_list7,
               [item for item in feature_list8]]
    featureUniqueList = np.unique([item for sublist in featureList for item in sublist])
    featureDictionary = dict.fromkeys(featureUniqueList)
    for key in featureDictionary.keys():
        featureDictionary[key] = []
    

    for feature in featureUniqueList:
    
        for i, method in enumerate(methodList):
        
            if feature in featureList[i]:
                featureDictionary[feature].append(method)   

    featureDf = pd.DataFrame(index=featureUniqueList, columns=methodList)
    featureDf['Count'] = 0

    for feature in featureDictionary.keys(): 
        for method in methodList:
        
            if method in featureDictionary[feature]:
            
                featureDf['Count'][feature] += 1
            
                featureDf[method][feature] = 'X'
            else:
                featureDf[method][feature] = '-' 
    featureDf.sort_values(by='Count', ascending=False, inplace=True)

    #Classification RF
    pipeRF = Pipeline([('classifier', [RandomForestClassifier()])])
    param_grid = [
    {'classifier' : [RandomForestClassifier()],
    'classifier__criterion':('gini','entropy'),
    'classifier__class_weight':('balanced','auto')}]
    clf = GridSearchCV(pipeRF, param_grid = param_grid, cv = cv, n_jobs=-1, scoring='f1_weighted')

    # Fit on data
    clf.fit(data_train[x_column_list],data_train[y_b])
    best_clf=clf.best_estimator_
    y_valid=best_clf.predict(data_val[x_column_list])
    matrix = confusion_matrix(data_val[y_b], y_valid)
    report_RF = classification_report(data_val[y_b],y_valid,output_dict=True)
    dRF = pd.DataFrame(report_RF).transpose()
    #Classification All Model
    pipe = Pipeline([('classifier', [RandomForestClassifier(),DecisionTreeClassifier(),SGDClassifier(),SVC(),KNeighborsClassifier()])])
    param_grid = [
    {'classifier' : [RandomForestClassifier()]},   
    {'classifier' : [DecisionTreeClassifier()]}, 
    {'classifier' : [SGDClassifier()]},
    {'classifier' : [SVC()],      
    'classifier__class_weight':('balanced','auto'),
    'classifier__gamma' : [1,0.1,0.01,0.001]},
    #'classifier__ kernel': ['rbf', 'poly', 'sigmoid']},
    {'classifier' : [KNeighborsClassifier()],
     'classifier__n_neighbors':list(range(5,25))} ]
    clf = GridSearchCV(pipe, param_grid = param_grid, cv = cv, n_jobs=-1, scoring='f1_weighted')
    # Fit on data
    clf.fit(data_train[x_column_list],data_train[y_b])
    best_clf=clf.best_estimator_
    y_valid=best_clf.predict(data_val[x_column_list])
    report_All = classification_report(data_val[y_b],y_valid,output_dict=True)
    dAll=pd.DataFrame(report_All).transpose()
    #Classification by selected feature
    selected_features = featureDf[featureDf['Count']>2].index
    
    clf = GridSearchCV(pipeRF, param_grid = param_grid, cv = cv, n_jobs=-1, scoring='f1_weighted')
    # Fit on data
    clf.fit(data_train[selected_features],data_train[y_b])
    best_clf=clf.best_estimator_
    y_valid=best_clf.predict(data_val[selected_features])
    report_selected = classification_report(data_val[y_b],y_valid,output_dict=True)
    dfselected1=pd.DataFrame(report_selected).transpose()



    ### from original data and selected features
    new_column_list = [col.replace('.','-') for col in selected_features]
    clf = GridSearchCV(pipeRF, param_grid = param_grid, cv = cv, n_jobs=-1, scoring='f1_weighted')
    # Fit on data
    clf.fit(data_original[new_column_list].div(data_original[new_column_list].sum(axis=1), axis=0)[0:len(data_train[y_b])],data_train[y_b])
    best_clf=clf.best_estimator_
    y_valid=best_clf.predict(data_original[new_column_list].div(data_original[new_column_list].sum(axis=1), axis=0)[len(data_train[y_b]):])
    report_selected2 = classification_report(data_val[y_b],y_valid,output_dict=True)
    dfselected2=pd.DataFrame(report_selected2).transpose()
    dfselected=pd.merge(dfselected1,dfselected2,on=dfselected2.index)

    #Clustering
    kmeans = KMeans(n_clusters=2, random_state=0).fit(data[x_column_list])
    report_k_means= classification_report(data[y_b],kmeans.labels_,output_dict=True)
    dK_means=pd.DataFrame(report_k_means).transpose()
    dK_means['key_0']=dK_means.index
    c_1=pd.merge(dRF,dAll,on=dRF.index)
    c_2=pd.merge(c_1,dK_means,on='key_0')
    c_3=pd.merge(c_2,dfselected,on='key_0')
    coln=['class','precision_RF','recall_RF','f1_RF','support_RF','precision_All','recall_All','f1_All','support_All','precision_k-m','recall_k-m','f1-score_k-m','support_k-m','precision_selected1','recall_selected1','f1-score_selected1','support_selected1','precision_selected2','recall_selected2','f1-score_selected2','support_selected2']
    c_3.columns=coln
    #####conclusion
    f1_cols = [col for col in c_3.columns if 'f1' in col]
    c_f1_t=c_3[f1_cols]
    c_f1=pd.DataFrame(c_f1_t.iloc[4,:])
    c_f1.rename(columns={4:'f1'}, inplace=True)
    namemethod=c_f1['f1'].idxmax()
    c_f1_t.index=dRF.index
    f_0=c_f1_t[namemethod][0]
    f_1=c_f1_t[namemethod][1]
    BestF=[f_0,f_1]
    F1_0_class_RF=c_3.iloc[0,3]
    F1_1_class_RF=c_3.iloc[1,3]
    return percent_label,featureDf, c_3,c_f1_t




In [9]:
df1=pd.read_csv("/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/pythoncode/Main Data Files/binary-data.csv")
df2 = pd.read_csv('/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/pythoncode/Main Data Files/metadata.csv')
df1=df1.drop('Variety2',axis=1)
df1.rename(columns={'Yield_per_plant':"y_binary"}, inplace=True)
df2 = df2[['Unnamed: 0', 'Yield_per_plant']]
df2.rename(columns={'Unnamed: 0':'Link_ID'}, inplace=True)
data_original = pd.merge(df1, df2, on='Link_ID')
cv=RepeatedKFold(n_splits=10,n_repeats=3, random_state=100)
th=0.05
NumofR=2
q=0.7
os.chdir('/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/pythoncode/Main Data Files/Netcomi_data')
path = '/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/pythoncode/Main Data Files/Netcomi_data'
file_list = []
for file in os.listdir(path):
    if (file[0] != 'R') & (file[-4:] == '.csv'):
        file_list.append(file[-4:]) 
output_dic = dict.fromkeys(file_list)
for file in os.listdir(path):
    if file[-4:] == '.csv':
        print(file)
        data_temp = pd.read_csv(file)
        output = process_data(data_original,data_temp,cv,q,NumofR)
        output_dic[file] = [output[0],output[1], output[2],output[3]]

 1 _ 3 .csv


/var/folders/96/yh0jxgf16t5_3vj4pfcbh3ph0000gn/T/ipykernel_21296/2114219913.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
/var/folders/96/yh0jxgf16t5_3vj4pfcbh3ph0000gn/T/ipykernel_21296/2114219913.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
/var/folders/96/yh0jxgf16t5_3vj4pfcbh3ph0000gn/T/ipykernel_21296/2114219913.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
/var/folders/96/yh0jxgf16t5_3vj4pfcbh3ph0000gn/T/ipykernel_21296/2114219913.py:6: FutureWarning: The default dtype for empty

Fitting 30 folds for each of 36 candidates, totalling 1080 fits


/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 21720.911192044616, tolerance: 6447.980305406523
  model = cd_fast.enet_coordinate_descent_gram(
/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8796.775257833302, tolerance: 6447.980305406523
  model = cd_fast.enet_coordinate_descent_gram(
/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22082.487622857094, tolerance: 6447.980305406523
  model = cd_fast.enet_coordinate_descent_gram(
/Users/rosa/opt/a

Fitting 30 folds for each of 36 candidates, totalling 1080 fits


...

/var/folders/96/yh0jxgf16t5_3vj4pfcbh3ph0000gn/T/ipykernel_21296/3529009864.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  featureDf[method][feature] = '-'
/var/folders/96/yh0jxgf16t5_3vj4pfcbh3ph0000gn/T/ipykernel_21296/3529009864.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  featureDf['Count'][feature] += 1
/var/folders/96/yh0jxgf16t5_3vj4pfcbh3ph0000gn/T/ipykernel_21296/3529009864.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

 4 _ 2 .csv


/var/folders/96/yh0jxgf16t5_3vj4pfcbh3ph0000gn/T/ipykernel_21296/2114219913.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
/var/folders/96/yh0jxgf16t5_3vj4pfcbh3ph0000gn/T/ipykernel_21296/2114219913.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
/var/folders/96/yh0jxgf16t5_3vj4pfcbh3ph0000gn/T/ipykernel_21296/2114219913.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
/var/folders/96/yh0jxgf16t5_3vj4pfcbh3ph0000gn/T/ipykernel_21296/2114219913.py:6: FutureWarning: The default dtype for empty

Fitting 30 folds for each of 36 candidates, totalling 1080 fits
Fitting 30 folds for each of 36 candidates, totalling 1080 fits


/var/folders/96/yh0jxgf16t5_3vj4pfcbh3ph0000gn/T/ipykernel_21296/3529009864.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  featureDf[method][feature] = '-'
/var/folders/96/yh0jxgf16t5_3vj4pfcbh3ph0000gn/T/ipykernel_21296/3529009864.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  featureDf['Count'][feature] += 1
/var/folders/96/yh0jxgf16t5_3vj4pfcbh3ph0000gn/T/ipykernel_21296/3529009864.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

In [10]:
#output_dic.items()
tRF=pd.DataFrame()
tALL=pd.DataFrame()
k=0
for key in output_dic.keys():
    if key != '.csv':
        print(key)
        tRF[k]=pd.DataFrame(output_dic[key][3]['f1_RF'].values)
        tALL[k]=pd.DataFrame(output_dic[key][3]['f1_All'].values)
        k=k+1
tRF.index=output_dic[key][3].index
tALL.index=output_dic[key][3].index
tRF.to_csv('/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Report/Normalization/totalF1_RFfor_randomizationmethods.csv')
tALL.to_csv('/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Report/Normalization/totalF1_ALLfor_randomizationmethods.csv')
       

 1 _ 3 .csv
 4 _ 1 .csv
 5 _ 1 .csv
 1 _ 2 .csv
 1 _ 4 .csv
 3 _ 1 .csv
 2 _ 1 .csv
 2 _ 2 .csv
 3 _ 2 .csv
 4 _ 4 .csv
 5 _ 4 .csv
 2 _ 3 .csv
 3 _ 3 .csv
 1 _ 1 .csv
 3 _ 4 .csv
 2 _ 4 .csv
 5 _ 3 .csv
 4 _ 3 .csv
 5 _ 2 .csv
 4 _ 2 .csv


In [11]:
for key in output_dic.keys():
    if key != '.csv':
        output_dic[key][1].to_csv('/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Report/Normalization/feature selection table '+key)

In [12]:
for key in output_dic.keys():
    if key != '.csv':
        output_dic[key][3].to_csv('/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Report/Normalization/F1-score'+key)

In [ ]:
k